In [1]:
from ovito.io import *
from ovito.modifiers import *
from ovito.data import *
from ovito.pipeline import *
import operator
import numpy as np
from scipy import integrate
from scipy.constants import codata

kb = codata.value('Boltzmann constant')
ev = codata.value('electron volt')
ev = -ev

# Ingore the deprecated warning
import warnings
warnings.filterwarnings("ignore")

/tmp/ipykernel_220937/3941677485.py:10: DeprecationWarning: Please use `value` from the `scipy.constants` namespace, the `scipy.constants.codata` namespace is deprecated.
  kb = codata.value('Boltzmann constant')
/tmp/ipykernel_220937/3941677485.py:11: DeprecationWarning: Please use `value` from the `scipy.constants` namespace, the `scipy.constants.codata` namespace is deprecated.
  ev = codata.value('electron volt')


In [2]:
def na_density_na(path_line = "/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp02/00-minimize/01-heat-773/02-diffusion/structure.dump.*", id=1, bin_x=700, x=False, y=False):

    pipeline = import_file(path_line)

    def setup_particle_types(frame, data):
                types = data.particles_.particle_types_
                types.type_by_id_(1).name = "O"
                types.type_by_id_(2).name = "Na"
                types.type_by_id_(3).name = "Zr"
                types.type_by_id_(4).name = "Si"
                types.type_by_id_(5).name = "P"
                #types.type_by_id_(5).radius = 0.3
        
    pipeline.modifiers.append(setup_particle_types)

    # Wrap at periodic boundaries:
    pipeline.modifiers.append(WrapPeriodicImagesModifier())

    #if type == True:
    id = id

    pipeline.modifiers.append(SelectTypeModifier(types = {id}))

    # Spatial binning:

    if x == True:
        pipeline.modifiers.append(SpatialBinningModifier(
            property = 'Selection', 
            reduction_operation = SpatialBinningModifier.Operation.Sum, 
            direction = SpatialBinningModifier.Direction.X, 
            bin_count = (bin_x, 80, 46)))
        data = pipeline.compute()
        cell = data.cell[:,0][0]
    elif y == True:
        pipeline.modifiers.append(SpatialBinningModifier(
            property = 'Selection', 
            reduction_operation = SpatialBinningModifier.Operation.Sum, 
            direction = SpatialBinningModifier.Direction.Y, 
            bin_count = (bin_x, 80, 46)))
        data = pipeline.compute()
        cell = data.cell[:,1][1]
    else:
        pipeline.modifiers.append(SpatialBinningModifier(
            property = 'Selection', 
            reduction_operation = SpatialBinningModifier.Operation.Sum, 
            direction = SpatialBinningModifier.Direction.Z, 
            bin_count = (bin_x, 80, 46)))
        data = pipeline.compute()
        cell = data.cell[:,2][2]
        
    
    # Time averaging:
    #pipeline.modifiers.append(TimeAveragingModifier(operate_on = 'table:binning'))

    charge_density = []

    for i in range(pipeline.source.num_frames):

        # Calculate electrostatic potential:
        data = pipeline.compute(frame=i)
        charge_density.append(data.tables['binning']['Selection'])

    charge_density = np.average(charge_density, axis=0)
    
    return charge_density, np.linspace(0,cell,bin_x)

In [3]:
def na_density_na_1(path_line = "/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp02/00-minimize/01-heat-773/02-diffusion/structure.dump.*", id=1, bin_x=700, x=False, y=False):

    pipeline = import_file(path_line)

    def setup_particle_types(frame, data):
                types = data.particles_.particle_types_
                types.type_by_id_(1).name = "Zr"
                types.type_by_id_(2).name = "Si"
                types.type_by_id_(3).name = "P"
                types.type_by_id_(4).name = "Na"
                types.type_by_id_(5).name = "O"
                types.type_by_id_(5).radius = 0.3
        
    pipeline.modifiers.append(setup_particle_types)

    # Wrap at periodic boundaries:
    pipeline.modifiers.append(WrapPeriodicImagesModifier())

    #if type == True:
    id = id

    pipeline.modifiers.append(SelectTypeModifier(types = {id}))

    # Spatial binning:

    if x == True:
        pipeline.modifiers.append(SpatialBinningModifier(
            property = 'Selection', 
            reduction_operation = SpatialBinningModifier.Operation.Sum, 
            direction = SpatialBinningModifier.Direction.X, 
            bin_count = (bin_x, 80, 46)))
        data = pipeline.compute()
        cell = data.cell[:,0][0]
    elif y == True:
        pipeline.modifiers.append(SpatialBinningModifier(
            property = 'Selection', 
            reduction_operation = SpatialBinningModifier.Operation.Sum, 
            direction = SpatialBinningModifier.Direction.Y, 
            bin_count = (bin_x, 80, 46)))
        data = pipeline.compute()
        cell = data.cell[:,1][1]
    else:
        pipeline.modifiers.append(SpatialBinningModifier(
            property = 'Selection', 
            reduction_operation = SpatialBinningModifier.Operation.Sum, 
            direction = SpatialBinningModifier.Direction.Z, 
            bin_count = (bin_x, 80, 46)))
        data = pipeline.compute()
        cell = data.cell[:,2][2]
        
    
    # Time averaging:
    #pipeline.modifiers.append(TimeAveragingModifier(operate_on = 'table:binning'))

    charge_density = []

    for i in range(pipeline.source.num_frames):

        # Calculate electrostatic potential:
        data = pipeline.compute(frame=i)
        charge_density.append(data.tables['binning']['Selection'])

    charge_density = np.average(charge_density, axis=0)
    
    return charge_density, np.linspace(0,cell,bin_x)

In [4]:
# GB, sigma 7 (180)
zr_dens, dat = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp02/00-minimize/01-heat-623/02-diffusion/trajectory/structure.dump.*",id=1, bin_x=500, x=False, y=False)
si_dens, dat = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp02/00-minimize/01-heat-623/02-diffusion/trajectory/structure.dump.*",id=2, bin_x=500, x=False, y=False)
p_dens, dat = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp02/00-minimize/01-heat-623/02-diffusion/trajectory/structure.dump.*",id=3, bin_x=500, x=False, y=False)
na_dens, dat = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp02/00-minimize/01-heat-623/02-diffusion/trajectory/structure.dump.*",id=4, bin_x=500, x=False, y=False)
o_dens, dat = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp02/00-minimize/01-heat-623/02-diffusion/trajectory/structure.dump.*",id=5, bin_x=500, x=False, y=False)

In [5]:
na_dens_523, dat = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp02/00-minimize/01-heat-523/02-diffusion/trajectory/structure.dump.*",id=4, bin_x=500, x=False, y=False)
na_dens_623, dat = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp02/00-minimize/01-heat-623/02-diffusion/trajectory/structure.dump.*",id=4, bin_x=500, x=False, y=False)
na_dens_773, dat = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp02/00-minimize/01-heat-773/02-diffusion/trajectory/structure.dump.*",id=4, bin_x=500, x=False, y=False)

In [6]:
# Sigma 11 (41)
zr_dens_06, dat_06 = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp06/00-minimize/01-heat-623/02-diffusion/structure.dump.*",id=1, bin_x=500, x=True, y=False)
si_dens_06, dat_06 = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp06/00-minimize/01-heat-623/02-diffusion/structure.dump.*",id=2, bin_x=500, x=True, y=False)
p_dens_06, dat_06 = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp06/00-minimize/01-heat-623/02-diffusion/structure.dump.*",id=3, bin_x=500, x=True, y=False)
na_dens_06, dat_06 = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp06/00-minimize/01-heat-623/02-diffusion/structure.dump.*",id=4, bin_x=500, x=True, y=False)
o_dens_06, dat_06 = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp06/00-minimize/01-heat-623/02-diffusion/structure.dump.*",id=5, bin_x=500, x=True, y=False)

In [ ]:
na_dens_06_523, dat_06_523 = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp06/00-minimize/01-heat-523/02-diffusion/structure.dump.*",id=4, bin_x=500, x=True, y=False)
na_dens_06_623, dat_06_623 = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp06/00-minimize/01-heat-623/02-diffusion/structure.dump.*",id=4, bin_x=500, x=True, y=False)
na_dens_06_773, dat_06_773 = na_density_na_1(path_line="/nfshome/deshmukh/vaibhav/schulze_project/gb/IDp06/00-minimize/01-heat-773/02-diffusion/structure.dump.*",id=4, bin_x=500, x=True, y=False)

In [ ]:
# GB 5 120, at 623 K

zr_density_5, dat_5 = na_density_na(path_line = "/nfshome/deshmukh/vaibhav/schulze_project/gb/gb_3/relaxation_2/gb_5_120/01-heat-623/02-diffusion/structure.dump.*", id=3, bin_x=500, x=False, y=True)
si_density_5, dat_5 = na_density_na(path_line = "/nfshome/deshmukh/vaibhav/schulze_project/gb/gb_3/relaxation_2/gb_5_120/01-heat-623/02-diffusion/structure.dump.*", id=4, bin_x=500, x=False, y=True)
p_density_5, dat_5 = na_density_na(path_line = "/nfshome/deshmukh/vaibhav/schulze_project/gb/gb_3/relaxation_2/gb_5_120/01-heat-623/02-diffusion/structure.dump.*", id=5, bin_x=500, x=False, y=True)
na_density_5, dat_5 = na_density_na(path_line = "/nfshome/deshmukh/vaibhav/schulze_project/gb/gb_3/relaxation_2/gb_5_120/01-heat-623/02-diffusion/structure.dump.*", id=2, bin_x=500, x=False, y=True)
o_density_5, dat_5 = na_density_na(path_line = "/nfshome/deshmukh/vaibhav/schulze_project/gb/gb_3/relaxation_2/gb_5_120/01-heat-623/02-diffusion/structure.dump.*", id=1, bin_x=500, x=False, y=True)

In [6]:
def centered_running_mean(data, window_size):
    pad_size = window_size // 2
    # Padding the data
    padded_data = np.pad(data, (pad_size, pad_size), 'edge')
    cumsum = np.cumsum(np.insert(padded_data, 0, 0))
    centered_mean = (cumsum[window_size:] - cumsum[:-window_size]) / float(window_size)
    
    # Return the centered mean, aligned with the original data length
    return centered_mean